In [1]:
from pathlib import Path

import numpy as np
import pandas as pd

from dsgrid.dataformat.datafile import Datafile
from dsgrid.dataformat.datatable import Datatable

from ntbkhelp import OptionPresenter, show_enum, show_elements_with_data

dsgrid_dataset_path = Path("//nrelnas01/PLEXOS/Projects/Load/dsgrid_v0.2.0/data")

## Choose a file

In [2]:
files = [p.name for p in dsgrid_dataset_path.glob("*.dsg")]
options = OptionPresenter(files)
options.present_options()
input_str = input("Which file would you like to examine? ")
datafile = Datafile.load(dsgrid_dataset_path / options.get_option(input_str))

  1: chp_dg.dsg
  2: commercial.dsg
  3: commercial_gaps.dsg
  4: distributedpv_sectoral.dsg
  5: eia_annual_energy_by_sector.dsg
  6: eia_annual_losses.dsg
  7: historical_hourly_load.dsg
  8: industrial.dsg
  9: industrial_gaps.dsg
 10: loss_model.dsg
 11: municipal_water.dsg
 12: outdoor_lighting.dsg
 13: residential.dsg
 14: residential_gaps.dsg
 15: trans_rail_hourly.dsg
Which file would you like to examine? 2


## Examine Basic Information

In [ ]:
show_enum(datafile.sector_enum)

In [ ]:
show_elements_with_data(datafile.sector_enum, list(datafile.keys()))

In [ ]:
show_enum(datafile.geo_enum)

In [ ]:
# WARNING: This cell may take a few minutes to run if the geography is counties
geo_ids = []
for _id in datafile.geo_enum.ids:
    for sector_id in datafile:
        if datafile[sector_id].has_data(_id):
            geo_ids.append(_id)
            break
show_elements_with_data(datafile.geo_enum, geo_ids)

In [ ]:
show_enum(datafile.enduse_enum)

In [ ]:
for sector_id in datafile:
    df, _geo_ids, _scaling_factors = datafile[sector_id].get_data(0)
    break
show_elements_with_data(datafile.enduse_enum, list(df.columns))

In [ ]:
show_enum(datafile.time_enum)

## Get Total Energy per Sector and Enduse with Datatable

In [ ]:
# WARNING: This cell may be slow and/or run out of memory for large files / county level data (e.g., 
# commercial.dsg, residential.dsg)
data_series = Datatable(datafile).data
total_energy = data_series.reset_index().pivot_table(values=0, index="sector", columns="enduse", aggfunc=np.sum)
total_energy

## Get Total Energy per Sector and Enduse by Iterating

In [3]:
total_energy = []; index = []
for sector_id in datafile:
    sector_dataset = datafile[sector_id]
    sector_energy = None
    for i in range(sector_dataset.n_geos):
        df, geos, scaling_factors = sector_dataset.get_data(i)
        tmp = df.sum()
        for factor in scaling_factors:
            if sector_energy is None:
                sector_energy = (tmp * factor)
            else:
                sector_energy += (tmp * factor)
    if sector_energy is not None:
        total_energy.append(sector_energy.to_frame().T)
        index.append(sector_id)
total_energy = pd.concat(total_energy)
total_energy.index = index
total_energy

,fans,pumps,heating,cooling,interior_lights,exterior_lights,water_systems,interior_equipment,heat_rejection
com__FullServiceRestaurant,5.592026e+06,5.593092e+04,9.139298e+05,4.633616e+06,9.906302e+06,2.767272e+06,0.0,2.221512e+07,2.024623e+04
com__Hospital,4.455086e+06,7.751797e+05,7.549169e+03,4.685746e+06,6.702352e+06,5.133133e+05,0.0,8.874651e+06,3.591725e+05
com__LargeHotel,8.412271e+06,4.632291e+05,4.993942e+05,7.993656e+06,9.188369e+06,2.308821e+06,0.0,1.467052e+07,1.641594e+05
com__LargeOffice,6.120055e+07,7.537129e+06,1.341884e+06,5.354156e+07,7.956575e+07,1.616527e+07,0.0,9.237318e+07,2.822176e+06
com__MediumOffice,1.436847e+07,7.885793e+05,7.406976e+05,1.138845e+07,1.948978e+07,6.934036e+06,0.0,2.223694e+07,2.750128e+05
com__Outpatient,1.503822e+06,1.695997e+05,0.000000e+00,9.865392e+05,3.726250e+06,1.334083e+06,0.0,5.502770e+06,7.595111e+04
com__PrimarySchool,2.240584e+06,7.669977e+04,4.206559e+05,1.870952e+06,4.758928e+06,6.122908e+05,0.0,3.796403e+06,3.673554e+04
com__QuickServiceRestaurant,1.740887e+05,1.145789e+03,4.317019e+03,1.255920e+05,8.780455e+04,5.153893e+04,0.0,5.210781e+05,8.612583e+02
com__SmallHotel,1.466056e+05,1.387005e+04,8.728184e+04,1.118608e+05,1.074873e+06,7.157537e+05,0.0,1.508302e+06,5.497838e+03
com__SmallOffice,1.089325e+07,8.049486e+04,1.424234e+06,8.286468e+06,1.596078e+07,6.235034e+06,0.0,2.186099e+07,4.979834e+04


In [4]:
total_energy.sum().sum() / 1.0E6

1107.037184